In [3]:
import pandas as pd
import os

In [4]:
map_mes = {
    1.0: 'janeiro',
    2.0: 'fevereiro',
    3.0: 'março',
    4.0: 'abril',
    5.0: 'maio',
    6.0: 'junho',
    7.0: 'julho',
    8.0: 'agosto',
    9.0: 'setembro',
    10.0: 'outubro',
    11.0: 'novembro',
    12.0: 'dezembro'
}

In [5]:
path_ = '../input/dados_html/'

all_files = os.listdir(path_)

all_dfs = []

for file in all_files:
    if file.split(".")[-1] == 'html':
        df_ = pd.read_html(path_ + file, encoding='utf-8', thousands=".")[0]
        df_ = df_[['Ano','Mês','Total mensal']]
        df_ = df_[:-1]
        all_dfs.append(df_)

    
all_data = pd.concat(all_dfs, ignore_index=True)
all_data = all_data.sort_values(by=['Ano','Mês'])
all_data.reset_index(inplace=True, drop=True)
all_data['Mês'] = all_data['Mês'].map(map_mes)
all_data.rename(columns={"Ano": 'ano', 'Mês': 'mes', "Total mensal": 'total'}, inplace=True)
all_data

,ano,mes,total
0,2003,janeiro,6500
1,2003,fevereiro,2370
2,2003,março,3958
3,2003,abril,3431
4,2003,maio,11297
...,...,...,...
223,2021,agosto,61931
224,2021,setembro,96260
225,2021,outubro,37579
226,2021,novembro,15357


In [6]:
all_data['total_acumulado'] = all_data.groupby(by=['ano'])['total'].cumsum()
all_data

,ano,mes,total,total_acumulado
0,2003,janeiro,6500,6500
1,2003,fevereiro,2370,8870
2,2003,março,3958,12828
3,2003,abril,3431,16259
4,2003,maio,11297,27556
...,...,...,...,...
223,2021,agosto,61931,120979
224,2021,setembro,96260,217239
225,2021,outubro,37579,254818
226,2021,novembro,15357,270175


In [7]:
all_data_sum = all_data.groupby(by='ano').sum()
all_data_sum.reset_index(inplace=True)
all_data_sum = all_data_sum[['ano','total']]
all_data_sum

,ano,total
0,2003,422362
1,2004,433058
2,2005,456873
3,2006,294178
4,2007,588387
5,2008,302329
6,2009,193014
7,2010,510116
8,2011,227750
9,2012,391800


In [6]:
all_data.to_csv("../output/area_queimada_por_ano_mes.csv", index=False)

In [7]:
all_data_sum.to_csv("../output/area_queimada_por_ano.csv", index=False)

In [8]:
path_ = '../input/dados_html/'

all_files = os.listdir(path_)

all_dfs_bioma = []

for file in all_files:
    if file.split(".")[-1] == 'html':
        df_ = pd.read_html(path_ + file, encoding='utf-8', thousands=".")[0]
        df_ = df_[:-1]
        all_dfs_bioma.append(df_)

    
all_data_bioma = pd.concat(all_dfs_bioma, ignore_index=True)
all_data_bioma = all_data_bioma.sort_values(by=['Ano','Mês'])
all_data_bioma.reset_index(inplace=True, drop=True)
all_data_bioma['mes_nome'] = all_data_bioma['Mês'].map(map_mes)
all_data_bioma['Mês'] = all_data_bioma['Mês'].astype("int")
all_data_bioma.rename(columns={"Ano": 'ano', 'Mês': 'mes', "Total mensal": 'total'}, inplace=True)
all_data_bioma.rename(columns={"Amazônia": 'amazonia', 'Caatinga': 'caatinga', "Cerrado": 'cerrado',
                              "Mata Atlântica": 'mata_atlantica', 'Pampa': 'pampa', 'Pantanal': 'pantanal'}, inplace=True)
all_data_bioma

,ano,mes,amazonia,caatinga,cerrado,mata_atlantica,pampa,pantanal,total,mes_nome
0,2003,1,3528,2206,157,436,1.0,172.0,6500,janeiro
1,2003,2,1960,56,214,107,3.0,30.0,2370,fevereiro
2,2003,3,3295,80,258,286,1.0,38.0,3958,março
3,2003,4,1179,440,1322,421,13.0,56.0,3431,abril
4,2003,5,2033,248,5200,3321,316.0,179.0,11297,maio
...,...,...,...,...,...,...,...,...,...,...
223,2021,8,18560,3300,32070,4682,123.0,3196.0,61931,agosto
224,2021,9,10045,13604,57473,6775,9.0,8354.0,96260,setembro
225,2021,10,5840,12722,12799,1272,15.0,4931.0,37579,outubro
226,2021,11,2148,12151,438,272,30.0,318.0,15357,novembro


In [9]:
all_data_bioma['amazonia_acumulado'] = all_data_bioma.groupby(by=['ano'])['amazonia'].cumsum()
all_data_bioma['caatinga_acumulado'] = all_data_bioma.groupby(by=['ano'])['caatinga'].cumsum()
all_data_bioma['cerrado_acumulado'] = all_data_bioma.groupby(by=['ano'])['cerrado'].cumsum()
all_data_bioma['mata_atlantica_acumulado'] = all_data_bioma.groupby(by=['ano'])['mata_atlantica'].cumsum()
all_data_bioma['pampa_acumulado'] = all_data_bioma.groupby(by=['ano'])['pampa'].cumsum()
all_data_bioma['pantanal_acumulado'] = all_data_bioma.groupby(by=['ano'])['pantanal'].cumsum()
all_data_bioma = all_data_bioma[['ano','mes','mes_nome','amazonia',
                                 'amazonia_acumulado','caatinga','caatinga_acumulado','cerrado',  'cerrado_acumulado', 'mata_atlantica',
                                 'mata_atlantica_acumulado','pampa','pampa_acumulado','pantanal','pantanal_acumulado','total']]
all_data_bioma

,ano,mes,mes_nome,amazonia,amazonia_acumulado,caatinga,caatinga_acumulado,cerrado,cerrado_acumulado,mata_atlantica,mata_atlantica_acumulado,pampa,pampa_acumulado,pantanal,pantanal_acumulado,total
0,2003,1,janeiro,3528,3528,2206,2206,157,157,436,436,1.0,1.0,172.0,172.0,6500
1,2003,2,fevereiro,1960,5488,56,2262,214,371,107,543,3.0,4.0,30.0,202.0,2370
2,2003,3,março,3295,8783,80,2342,258,629,286,829,1.0,5.0,38.0,240.0,3958
3,2003,4,abril,1179,9962,440,2782,1322,1951,421,1250,13.0,18.0,56.0,296.0,3431
4,2003,5,maio,2033,11995,248,3030,5200,7151,3321,4571,316.0,334.0,179.0,475.0,11297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,2021,8,agosto,18560,26893,3300,8678,32070,66548,4682,12428,123.0,1141.0,3196.0,5291.0,61931
224,2021,9,setembro,10045,36938,13604,22282,57473,124021,6775,19203,9.0,1150.0,8354.0,13645.0,96260
225,2021,10,outubro,5840,42778,12722,35004,12799,136820,1272,20475,15.0,1165.0,4931.0,18576.0,37579
226,2021,11,novembro,2148,44926,12151,47155,438,137258,272,20747,30.0,1195.0,318.0,18894.0,15357


In [13]:
bioma_cut = all_data_bioma[['ano','mes','mes_nome','amazonia_acumulado','caatinga_acumulado','cerrado_acumulado','mata_atlantica_acumulado','pampa_acumulado','pantanal_acumulado']]
bioma_cut.rename(columns={
    'amazonia_acumulado': 'amazonia',
    'caatinga_acumulado': 'caatinga',
    'cerrado_acumulado': 'cerrado',
    'mata_atlantica_acumulado': 'mata_atlantica',
    'pampa_acumulado': 'pampa',
    'pantanal_acumulado': 'pantanal'}, inplace=True)
bioma_cut = bioma_cut.melt(id_vars=['ano','mes','mes_nome'], value_vars=['amazonia','caatinga','cerrado','mata_atlantica','pantanal','pampa'])
bioma_cut.rename(columns={'variable': 'bioma', 'value': 'total'}, inplace=True)
bioma_cut

/Users/adolfoguimaraes/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,ano,mes,mes_nome,bioma,total
0,2003,1,janeiro,amazonia,3528.0
1,2003,2,fevereiro,amazonia,5488.0
2,2003,3,março,amazonia,8783.0
3,2003,4,abril,amazonia,9962.0
4,2003,5,maio,amazonia,11995.0
...,...,...,...,...,...
1363,2021,8,agosto,pampa,1141.0
1364,2021,9,setembro,pampa,1150.0
1365,2021,10,outubro,pampa,1165.0
1366,2021,11,novembro,pampa,1195.0


In [14]:
all_data_bioma.to_csv("../output/area_queimada_por_ano_mes_bioma.csv", index=False)

In [15]:
bioma_cut.to_csv("../output/area_queimada_por_ano_mes_bioma_colum.csv", index=False)